In [1]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

import numpy as np
print(np.__version__)

from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking
import os
import pickle

1.26.4


c:\Users\Bulkin\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
chunks = RAMChunkStorage()
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)

algorithm = NatashaSentenceChunking(raw_storage=documents, chunk_storage=chunks, sentences_in_chunk=3)
algorithm.execute(visualize=True)

1

In [3]:
index = chunks.get_indexes()[18]
chunks[index]

'Приходится признать, что ни одна из этих сводок никуда не годится. Раньше всего: ни на какую ногу описываемый не хромал, и росту был не маленького и не громадного, а просто высокого. Что касается зубов, то с левой стороны у него были платиновые коронки, а с правой — золотые.'

In [4]:
print(
    sum(
        chunks[index] == "Речь эта, как впоследствии узнали, шла об Иисусе Христе." for index in chunks.get_indexes()
    )
)

0


In [5]:
from mirage.embedders.BowEmbedder import BowEmbedder

emb = HuggingFaceEmbedder(normalizer = True)
emb.fit(chunks)

In [6]:
emb.get_dimensionality()

768

In [10]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexIVFPQR(dimensionality=emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 2833/2833 [02:18<00:00, 20.51it/s]


In [11]:
query = 'начальник Иудеи утвердил смертный приговор'
query_vector = emb.embed(text=query)
# print(query_vector)
answer = vector_index.query(
    query_vector, top_k=20
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[21.82748794555664, 27.86431884765625, 27.915950775146484, 29.267833709716797, 30.334896087646484, 30.927810668945312, 33.16828536987305, 33.2599983215332, 33.84587860107422, 34.197486877441406, 34.40011215209961, 34.450294494628906, 35.88490295410156, 36.218040466308594, 37.174537658691406, 37.280086517333984, 38.33465576171875, 38.376556396484375, 39.9251823425293, 40.35816192626953]


['—\xa0Напрасно! Так не прикажете ли партию в кости? Или вы предпочитаете другие какие-нибудь игры?',
 'Наташа подкуплена? да? Но как вы могли узнать мои мысли? — она страдальчески сморщилась и добавила: — Скажите мне, кто вы такой? Из какого вы учреждения?',
 'Как же. Непременно. Срочно.',
 'Некий человек… —\xa0Левий Матвей, — не вопросительно, а скорее утвердительно сказал Пилат. —\xa0Да, прокуратор…',
 '—\xa0Да нету его дома. Я уже Карпова посылал. Никого нету в квартире.',
 'Только не напрягайте мозг. Не выйдет сегодня, выйдет завтра. —\xa0Он уйдет!',
 'А я-то на вас надеялся. Итак, номер наш не удался. В зале раздался свист по адресу Никанора Ивановича.',
 '—\xa0Ну, что же, теперь, я надеюсь, вы вспомнили мою фамилию? Но Степа только стыдливо улыбнулся и развел руками. —\xa0Однако!',
 'Напротив еще палисадничек? Как же, знаю, знаю! А куда ж вы их там засунули?',
 'Ну, натурально, я выходил гулять. Сто тысяч — громадная сумма, и у меня был прекрасный серый костюм. Или отправлялся о

In [9]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               promt='')

ModuleNotFoundError: No module named 'pydantic'